# Workflow for Par version of Neo-2

## WARNING: This is a template, do not edit/use it in the neo-2 code folder. Make a copy to the folder where you want to work (e.g. /temp/_username_/), and use that one.

Example notebook to demonstrate/teach basic usage of the Python code and a basic workflow on how to get/plot data.

This is thought as addition to the '2018_Documentation.pdf', not as a replacement.

Here we will not start a single run, but a whole scan over radius.

Before starting:
- check out the code from github (acount required) if you have not already.
- set the environment variable 'NEO2PATH' to your neo-2 directory in .bashrc (or equivalent of your system).

This first cell makes sure that the python scripts of the neo2 repository can be found. This is done by making use of the 'NEO2PATH' environment variable.

_HINT: those cells with a 'In' next to it, are input cells, to execute one, first activate the cell, e.g. by clicking on it, and then press **shift+enter**.
Another possibility is to use the entry **Cell -> Run Cells** in the menu above_

In [ ]:
import sys
import os
try:
  sys.path.append(os.path.join(os.environ['NEO2PATH'], 'PythonScripts'))
except KeyError:
  print("You need to set NEO2PATH in your .bashrc, e.g. export NEO2PATH=$HOME/neo-2/")

The second cell imports the modules/functions/classes we will use.
This is done in one place to have a good overview of dependencies.
For the same reason module names are retained: so it is easy to see where a command comes from.

In [ ]:
# System
import shutil
#import sys # this and os are already imported, as they are required for setting the path.
#import os

# Third party

# Own (neo2/libneo)
import create_surf
import neo2tools
import scan_nonhdf5_tools

In [ ]:
# if relative, then relative to path where notebook was started(?), only subdirectories allowed(?)
workingdirectory = '/temp/buchholz/Scans/Recreate_Gernots_Results/w7x_test2/s1.000000000m03/'
scan = neo2tools.Neo2Scan(wdir=workingdirectory, templatepath=None)

### Compiling of the code

For now we assume this has already been done, i.e. the compiled code is already available.

### Get required (input) files

For a Neo-2 run basic input files are:
- neo.in (exactly this name, settings file)
- boozer file (magnetic equilibrium)
- profile file
- neo2.in (exactly this name, namelist file)

Copy the first three files into the work folder.
Additionally you will need

- create_surfaces.in
- create_surfaces.x

(e.g. from /proj/plasma/Neo2/Interface/Create_Surfaces/). The former should be copied into the work folder, while the later might be linked.
Make sure that the first non-comment line in neo.in matches the name of the boozer file you use.

In the work directory create a subdirectory called 'TEMPLATE_DIR', which will be used to create the folders for the flux surfaces.
Into this folder copy the executable neo_2.x, neo2.in, the boozer file, and a copy of neo2.in.

Additionally you will need a configuration file for condor. An example is DOC/condor_submit_example, in the NEO2PATH.
Note that the approach in this example is different from those used in the '2018_Documentation.pdf'.

In [ ]:
os.mkdir(workingdirectory)
os.chdir(workingdirectory)
os.mkdir(os.path.join(workingdirectory, 'TEMPLATE_DIR'))

# Note: either uncomment the following lines and fill in the proper locations and names,
# or copy the files externaly, e.g. in a file browser.
#shutil.copy('REPLACE_ME/neo.in', './')
#shutil.copy('REPLACE_ME/boozer_file.bc', './')
#shutil.copy('REPLACE_ME/profile_file', './')

shutil.copy('/proj/plasma/Neo2/Interface/Create_Surfaces/create_surfaces.in', './')
shutil.copy('/proj/plasma/Neo2/Interface/Create_Surfaces/create_surfaces.x', './')

shutil.copy(os.path.join(os.environ['NEO2PATH'], 'DOC/condor_submit_example'), './submit')

### Create files and folders

First step is now to run create_surfaces.x

In [ ]:
os.system('./create_surfaces.x')

This interpolates the given input profile into the desired radial output locations (written to 'surfaces.dat'), as specified in create_surfaces.in.

The next step now actually creates the folders required for the radial scan.
This can be done with the python function create_surfaces(). This takes the output of the executable and the content from the 'TEMPLATE_DIR' folder to create a folder for each flux surface. The place holders in neo2.in are replaced with the actual values.

In [ ]:
create_surf.create_surfaces()

### Start simulations

All the folders and files are now created and have the required values. So we can now start the simulations.

In [ ]:
os.system('condor_submit submit')

Now we have to wait until the jobs are finished. Once this is the case we can continue here.

Unfortunately multiple runs are necessary, with the parameter 'prop_reconstruct' of the namelist propagator changed. There is a function to do this:

In [ ]:
scan_nonhdf5_tools.set_neo2in_reconstruction(folder='./',
                                             subfolder_pattern='s[123456789].*',
                                             backup=True,
                                             value=1)
os.system('condor_submit submit')

We are still not done, two more runs are required.

In [ ]:
scan_nonhdf5_tools.set_neo2in_reconstruction(folder='./',
                                             subfolder_pattern='s[123456789].*',
                                             backup=False,
                                             value=2)
os.system('condor_submit submit')

In [ ]:
scan_nonhdf5_tools.set_neo2in_reconstruction(folder='./',
                                             subfolder_pattern='s[123456789].*',
                                             backup=False,
                                             value=3)
os.system('condor_submit submit')

Now the scan is actually finished and we can check on the data.

_NOTE: some data may already be available after the first or the third run, but unless you know what you are doing stick to doing all four runs._

### Some data collecting

Here we will collect the data and make some preparations for plotting them.

Running of neo2_g.x falls into both categories. The program will calculate the distribution function for a given number of points and write these data to a single hdf5 file.

In [ ]:
# collection into a single final.h5?

if not os.path.isdir(os.path.join(workingdirectory, 'PLOTS')):
  os.mkdir(os.path.join(workingdirectory, 'PLOTS'))
os.chdir(os.path.join(workingdirectory, 'PLOTS'))

if not os.path.islink('neo2_g.x'):
  os.symlink('/proj/plasma/Neo2/Interface/Spitzer_Interface/Build/neo2_g.x', 'neo2_g.x')
#/proj/plasma/Neo2/Interface/Spitzer_Interface/Build/dentf_lorentz.x

if not os.path.isfile('g_vs_lambda.in'):
  shutil.copy('/proj/plasma/Neo2/Interface/Examples/g_vs_lambda.in', './')
if not os.path.isfile('spitzerinterface.in'):
  shutil.copy('/proj/plasma/Neo2/Interface/Examples/spitzerinterface.in', './')

os.system('./neo2_g.x')

### Plot results

Now the data can be plotted.
So we create an instance of the plotting class and use it to make a plot of the magnetic field line and of the distribution function.

In [ ]:
#/proj/plasma/DOCUMENTS/PHD_Gernot/2017_BOOTSTRAP/plot_evolve_h5.m
#/proj/plasma/Neo2/Interface/Matlab/plot_g_xfix.m
#plot_levels_pgf.m
#/proj/plasma/DOCUMENTS/PHD_Gernot/2017_BOOTSTRAP/
# count_maxima_bootstrap.m
rp = neo2tools.ReconPlot(plotdir='./PLOTS/', rundir=workingdirectory)
rp.magnetic_plot(2)
rp.g_plot('g', second_dimension_index='100')